In [1]:
import matplotlib
matplotlib.use('agg')  # allows notebook to be tested in Travis

import pandas as pd
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import pandana as pdna
import time

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

%matplotlib inline

In [2]:
feeds.add_feed(add_dict={'metro_bus': 'https://gitlab.com/LACMTA/gtfs_bus/raw/master/gtfs_bus.zip'})
feeds.add_feed(add_dict={'metro_rail': 'https://gitlab.com/LACMTA/gtfs_rail/raw/master/gtfs_rail.zip'})

Added 1 feeds to gtfs_feeds: {'metro_bus': 'https://gitlab.com/LACMTA/gtfs_bus/raw/master/gtfs_bus.zip'}
Added 1 feeds to gtfs_feeds: {'metro_rail': 'https://gitlab.com/LACMTA/gtfs_rail/raw/master/gtfs_rail.zip'}


In [3]:
gtfsfeeds.download()

2 GTFS feed(s) will be downloaded here: data\gtfsfeed_zips
metro_bus GTFS feed downloaded successfully. Took 6.25 seconds for 18,289,711.0KB
metro_rail GTFS feed downloaded successfully. Took 0.82 seconds for 660,824.0KB
GTFS feed download completed. Took 7.09 seconds
metro_bus.zip successfully extracted to: data\gtfsfeed_text\metro_bus
metro_rail.zip successfully extracted to: data\gtfsfeed_text\metro_rail
GTFS feed zipfile extraction completed. Took 1.92 seconds for 2 files


In [4]:
validation = True
verbose = True 

#bbox for project site
bbox = -118.617668,33.660954,-117.900962,34.207872
remove_stops_outsidebbox = True
append_definitions = True

loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path=None,
                                           validation=validation,
                                           verbose=verbose,
                                           bbox=bbox,
                                           remove_stops_outsidebbox=remove_stops_outsidebbox,
                                           append_definitions=append_definitions)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 0.85 seconds
--------------------------------
Processing GTFS feed: metro_bus
The unique agency id: metro_-_los_angeles was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.09 seconds
Unique GTFS feed id operation complete. Took 0.02 seconds
Records:
        stop_id  stop_code                    stop_name  stop_desc   stop_lat  \
20           33         33   Balboa / Foothill West Jog        NaN  34.322084   
26           47         47                 7th / Maclay        NaN  34.293980   
34           63         63            Agoura / La Venta        NaN  34.148950   
35           65         65          Agoura / Lost Hills        NaN  34.139410   
38           69         69    Lankershim / San Fernando        NaN  34.233180   
...         ...        ...                     

In [5]:
#create network for weekday AM peak
ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='monday',
                                   timerange=['07:00:00', '10:00:00'],
                                   calendar_dates_lookup=None)
urbanaccess_ampeak = ua.network.ua_network


Using calendar to extract service_ids to select trips.
23 service_ids were extracted from calendar
14,084 trip(s) 41.47 percent of 33,959 total trip records were found in calendar for GTFS feed(s): ['metro bus', 'metro rail']
NOTE: If you expected more trips to have been extracted and your GTFS feed(s) have a calendar_dates file, consider utilizing the calendar_dates_lookup parameter in order to add additional trips based on information inside of calendar_dates. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify service_ids. When in doubt do not use the calendar_dates_lookup parameter.
14,084 of 33,959 total trips were extracted representing calendar day: monday. Took 0.14 seconds
There are no departure time records missing from trips following the specified schedule. There are no records to interpolate.
Difference between stop times has been successfully calculated. Took 4.41 seconds
Stop times from 07:00:00 to 10:00

In [6]:
#load pedestrian OSM data for LA basin (takes several minutes) (adding remove_lcn = True adds ~35 minutes)
nodes, edges = ua.osm.load.ua_network_from_bbox(bbox=bbox, remove_lcn=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](33.66095400,-118.61766800,33.93494309,-118.25646126);>;);out;'}"
Downloaded 29,255.0KB from www.overpass-api.de in 15.81 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](33.93076790,-118.61766800,34.20839551,-118.26046406);>;);out;'}"
Downloaded 73,849.6KB from www.overpass-api.de in 36.06 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["ped

In [7]:
#create pedestrian-specific graph from OSM data
ua.osm.network.create_osm_net(osm_edges=edges,
                              osm_nodes=nodes,
                              travel_speed_mph=3)

Created OSM network with travel time impedance using a travel speed of 3 MPH. Took 0.04 seconds


In [8]:
#integrate OSM network with transit network
ua.network.integrate_network(urbanaccess_network=urbanaccess_ampeak,
                             headways=False)

Loaded UrbanAccess network components comprised of:
     Transit: 11,524 nodes and 117,925 edges;
     OSM: 519,177 nodes and 1,470,854 edges
Connector edges between the OSM and transit network nodes successfully completed. Took 9.35 seconds
Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 22.82 seconds
Network edge and node network integration completed successfully resulting in a total of 530,701 nodes and 1,611,827 edges:
     Transit: 11,524 nodes 117,925 edges;
     OSM: 519,177 nodes 1,470,854 edges; and
     OSM/Transit connector: 23,048 edges.


In [9]:
#save to disk
# ua.network.save_network(urbanaccess_network=urbanaccess_ampeak,
                        # filename='am/ampeak_net.h5')

In [10]:
#repeat process for off-peak workday
urbanaccess_midday = ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='monday',
                                   timerange=['11:00:00', '14:00:00'],
                                   calendar_dates_lookup=None)

Using calendar to extract service_ids to select trips.
23 service_ids were extracted from calendar
14,084 trip(s) 41.47 percent of 33,959 total trip records were found in calendar for GTFS feed(s): ['metro bus', 'metro rail']
NOTE: If you expected more trips to have been extracted and your GTFS feed(s) have a calendar_dates file, consider utilizing the calendar_dates_lookup parameter in order to add additional trips based on information inside of calendar_dates. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify service_ids. When in doubt do not use the calendar_dates_lookup parameter.
14,084 of 33,959 total trips were extracted representing calendar day: monday. Took 0.06 seconds
There are no departure time records missing from trips following the specified schedule. There are no records to interpolate.
Difference between stop times has been successfully calculated. Took 4.11 seconds
Stop times from 11:00:00 to 14:00

In [11]:
ua.network.integrate_network(urbanaccess_network=urbanaccess_midday,
                             headways=False)
# ua.network.save_network(urbanaccess_network=urbanaccess_midday,
                        # filename='midday_net.h5')

Loaded UrbanAccess network components comprised of:
     Transit: 11,272 nodes and 110,177 edges;
     OSM: 519,177 nodes and 1,470,854 edges
Connector edges between the OSM and transit network nodes successfully completed. Took 7.32 seconds
Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 19.59 seconds
Network edge and node network integration completed successfully resulting in a total of 530,449 nodes and 1,603,575 edges:
     Transit: 11,272 nodes 110,177 edges;
     OSM: 519,177 nodes 1,470,854 edges; and
     OSM/Transit connector: 22,544 edges.


In [12]:
urbanaccess_midday_weekend = ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='sunday',
                                   timerange=['11:00:00', '14:00:00'],
                                   calendar_dates_lookup=None)
ua.network.integrate_network(urbanaccess_network=urbanaccess_midday_weekend,
                             headways=False)
# ua.network.save_network(urbanaccess_network=urbanaccess_midday_weekend,
                        # filename='midday_weekend_net.h5')

Using calendar to extract service_ids to select trips.
17 service_ids were extracted from calendar
9,473 trip(s) 27.90 percent of 33,959 total trip records were found in calendar for GTFS feed(s): ['metro bus', 'metro rail']
NOTE: If you expected more trips to have been extracted and your GTFS feed(s) have a calendar_dates file, consider utilizing the calendar_dates_lookup parameter in order to add additional trips based on information inside of calendar_dates. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify service_ids. When in doubt do not use the calendar_dates_lookup parameter.
9,473 of 33,959 total trips were extracted representing calendar day: sunday. Took 0.06 seconds
There are no departure time records missing from trips following the specified schedule. There are no records to interpolate.
Difference between stop times has been successfully calculated. Took 2.69 seconds
Stop times from 11:00:00 to 14:00:0

In [13]:
urbanaccess_ampeak_weekend = ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='sunday',
                                   timerange=['07:00:00', '10:00:00'],
                                   calendar_dates_lookup=None)
ua.network.integrate_network(urbanaccess_network=urbanaccess_ampeak_weekend,
                             headways=False)
# ua.network.save_network(urbanaccess_network=urbanaccess_ampeak_weekend,
                        # filename='ampeak_weekend_net.h5')

Using calendar to extract service_ids to select trips.
17 service_ids were extracted from calendar
9,473 trip(s) 27.90 percent of 33,959 total trip records were found in calendar for GTFS feed(s): ['metro bus', 'metro rail']
NOTE: If you expected more trips to have been extracted and your GTFS feed(s) have a calendar_dates file, consider utilizing the calendar_dates_lookup parameter in order to add additional trips based on information inside of calendar_dates. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify service_ids. When in doubt do not use the calendar_dates_lookup parameter.
9,473 of 33,959 total trips were extracted representing calendar day: sunday. Took 0.06 seconds
There are no departure time records missing from trips following the specified schedule. There are no records to interpolate.
Difference between stop times has been successfully calculated. Took 2.70 seconds
Stop times from 07:00:00 to 10:00:0

In [14]:
import networkx as nx
G = nx.Graph()

In [15]:
import geopandas as gpd 
from shapely.geometry import Point, LineString, Polygon
from descartes import PolygonPatch
import matplotlib.pyplot as plt
import osmnx as ox 

trip_times = [15, 30, 45] #in minutes

In [16]:
import time
# import networkx as nx
import pandas as pd

def parse_nx_node(row, attr, idcol):
    """Parses nodes into nx-format"""
    attr = row[attr].copy()
    return (row[idcol], attr.to_dict())

def parse_nx_edge(row, attr_cols, fr, to):
    """Parses edges into nx-format"""
    idx = row.name
    attr = row[attr_cols].copy().to_dict()
    return (row[fr], row[to], idx, attr)


def optimize_graph(edges, aggr_type, fr='from_int', to='to_int', weight_col='weight'):
    """
    Optimize the graph in terms of keeping only e.g. 'min' or 'max' (fastest / slowest) edges between nodes.
    This process can dramatically decrease the size of the graph.
    Parameters
    ----------
    edges : pd.DataFrame
        Pandas DataFrame containing the edge information.
    aggr_type : str
        Aggregation type. Possible values are: 'min', 'max', 'mean', 'median'
    """
    # Group and aggregate edges by from and to -nodes using selected aggregation method
    optimized = edges.groupby([fr, to])[weight_col].agg(aggr_type).reset_index() \
        .merge(edges, on=[fr, to]).rename(columns={weight_col + '_x': weight_col}) \
        .drop_duplicates([fr, to]) \
        .drop(weight_col + '_y', axis=1)
    return optimized


def urbanaccess_to_nx(ua_G,
                      edge_attr=['from', 'to', 'from_int', 'to_int', 'weight', 'net_type', 'route_type',
                                 'sequence', 'unique_agency_id', 'unique_route_id',
                                 'unique_trip_id'],
                      crs={'init': 'epsg:4326'},
                      minimize=False,
                      use_integer_nodeids=True,
                      optimize=None,
                      weight_col='weight'
                      ):
    """
    Converts the input UrbanAccess graph to NetworkX MultiDiGraph.
    Parameters
    ----------
    ua_G : <urbanaccess.network.urbanaccess_network>
        UrbanAccess graph that should contain GTFS data as well as walking network, stored in net_edges & net_nodes.
    edge_attr : list
        Edge attributes that will be included in the final NetworkX graph. The necessary ones are 'from', 'to', and 'weight'.
    crs : CRS dictionary.
        Coordinate Reference System of the input data. GTFS and OSM are in WGS84 (epsg:4326) by default.
    minimize : <boolean> (default: False)
        If `minimize=True`, the tool will include only necessary attributes when creating the graph.
    use_integer_nodeids: Boolean (default: True)
        If `use_integer_nodeids=True`, the tool will build the graph using integer numbers as the nodeids.
    optimize : str (default: None)
        It is possible to optimize the network by including e.g. only fastest ('min') or longest ('max') journey legs between
        stops that might vary at different times of the day due schedule changes. Possible aggregation methods are: None, 'min', 'max', 'mean', 'median'.
    weight_col : str
        Name of the column that contains the cost information.
    Assumptions
    -----------
    The function assumes that the UrbanAccess graph is "full", i.e. the transit and walk networks have been built and integrated.
    To check this, you should have data inside DataFrames under `ua.network.ua_network.net_edges`, `ua.network.ua_network.net_nodes`,
    as well as in `ua.network.ua_network.net_connector_edges` (connects stops to walking network).
    """

    # Perform some basic sanity checks
    assert isinstance(ua_G.net_nodes, pd.DataFrame), "UrbanAccess network does not contain valid `net_nodes` DataFrame."
    assert isinstance(ua_G.net_edges, pd.DataFrame), "UrbanAccess network does not contain valid `net_edges` DataFrame."
    assert len(ua_G.net_nodes) > 0, "Could not find any nodes from UrbanAccess network."
    assert len(ua_G.net_edges) > 0, "Could not find any edges from UrbanAccess network."

    print("Converting to NetworkX graph ..")
    log_start = time.time()
    # Connected graph items
    nodes = ua_G.net_nodes
    edges = ua_G.net_edges
    graph = nx.MultiDiGraph()

    # Reset index for nodes to get the integer version of the node_id
    nodes = nodes.reset_index()

    if use_integer_nodeids:
        fr = 'from_int'
        to = 'to_int'
        nodeid = 'id_int'
    else:
        fr = 'from'
        to = 'to'
        nodeid = 'id'

    if minimize:
        exp_edge_attr = [fr, to, weight_col]
        exp_node_attr = [nodeid, 'x', 'y']
    else:
        exp_edge_attr = edge_attr
        exp_node_attr = list(nodes.columns)

    if optimize is not None:
        assert optimize in ['min', 'max', 'mean', 'median'], "Possible optimization methods are: 'min', 'max', 'mean', 'median'. Got %s." % optimize
        edges = optimize_graph(edges=edges, aggr_type=optimize, weight_col=weight_col)

    # Convert nodes
    start = time.time()
    nodes['nx_node'] = nodes.apply(parse_nx_node, attr=exp_node_attr, idcol=nodeid, axis=1)
    print("Converted nodes in %s minutes." % (round((time.time() - start) / 60, 1)))

    # Convert edges
    start = time.time()
    edges['nx_edge'] = edges.apply(parse_nx_edge, attr_cols=exp_edge_attr, fr=fr, to=to, axis=1)
    print("Converted edges in %s minutes." % (round((time.time() - start) / 60, 1)))

    # Create the Nx Graph
    graph.add_nodes_from(nodes['nx_node'].to_list())
    graph.add_edges_from(edges['nx_edge'].to_list())

    # Add metadata so that the graph works directly e.g. with OSMnx functions
    graph.graph['crs'] = crs
    graph.graph['name'] = edges['unique_agency_id'].unique()[0]
    print("Created NetworkX graph in %s minutes with %s nodes and %s edges." % (
    round((time.time() - log_start) / 60, 1), len(nodes), len(edges)))
    return graph

In [17]:
G = urbanaccess_to_nx(urbanaccess_ampeak)

Converting to NetworkX graph ..
Converted nodes in 5.7 minutes.
Converted edges in 18.4 minutes.
Created NetworkX graph in 24.5 minutes with 529544 nodes and 1571339 edges.


In [18]:
project_site = (33.9379368079315, -118.25653793837309)

center_node = ox.distance.get_nearest_node(G, project_site, method='haversine', return_dist=False)

In [19]:
mile_time = 0 #correct for mile circle

isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time + mile_time, distance='weight')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

In [22]:
G.graph['crs']

{'init': 'epsg:4326'}

In [24]:
from functools import partial

import pyproj
from shapely import geometry
from shapely.geometry import Point
from shapely.ops import transform

lon, lat = 33.93790489175171, -118.25653601748706 # lon lat for Lanzit site
radius = 1609  # in m

# local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(
#     lat, lon
# )
local_azimuthal_projection = 'epsg:4326'
wgs84_to_aeqd = partial(
    pyproj.transform,
    pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
    pyproj.Proj(local_azimuthal_projection),
)
aeqd_to_wgs84 = partial(
    pyproj.transform,
    pyproj.Proj(local_azimuthal_projection),
    pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
)

center = Point(float(lon), float(lat))
point_transformed = transform(wgs84_to_aeqd, center)
buffer = point_transformed.buffer(radius)
# Get the polygon with lat lon coordinates
circle_poly = transform(aeqd_to_wgs84, buffer)


In [ ]:
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0, return_hex=True)

# plot the network then add isochrones as colored descartes polygon patches
fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2, node_size=0, figsize=(12,8))
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.95, zorder=-1)
    ax.add_patch(patch)

# ax.add_patch(plt.Circle(center_UTM,1609,fill=None,ec='mediumpurple'))
ax.add_patch(PolygonPatch(circle_poly, fill=None,ec='mediumpurple'))

plt.savefig('figures/ampeak.png', bbox_inches='tight', dpi=300)
# plt.close()

In [34]:
import fiona 

# Enable fiona driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [48]:
gds = gpd.GeoSeries(isochrone_polys)
gds.to_file('transit_isochrone.kml',driver='KML')

In [42]:
def get_first_key(dictionary):
    for key in dictionary:
        return key
    raise IndexError

def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='weight')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            first = get_first_key(G.get_edge_data(n_fr, n_to))
            edge_lookup = G.get_edge_data(n_fr, n_to)[first].get('geometry',  LineString([f,t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union
        
        # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

# isochrone_polys_buffer = make_iso_polys(G, edge_buff=25, node_buff=50, infill=True)
